In [ ]:
#!curl https://sdk.cloud.google.com | bash

In [ ]:
!gcloud init

In [ ]:
!pip3 install --no-cache-dir -U crcmod

In [ ]:
!gsutil cp gs://paper-2-data/archive.zip .

Copying gs://paper-2-data/archive.zip...
/ [1 files][  3.8 GiB/  3.8 GiB]   70.2 MiB/s                                   
Operation completed over 1 objects/3.8 GiB.                                      


In [ ]:
import zipfile
with zipfile.ZipFile('archive.zip', 'r') as zip_ref:
    zip_ref.extractall('.')

In [ ]:
import pandas as pd
import numpy as np
from PIL import Image
import tensorflow as tf
import keras
import dask.dataframe as dd
import os

In [ ]:
np.set_printoptions(threshold=np.inf)
np.set_printoptions(linewidth=np.inf)

In [ ]:
df_train = dd.read_csv('train.csv', dtype={'id': 'object'})
df_val = dd.read_csv('valid.csv', dtype={'id': 'object'})

In [ ]:
labels = {'real':1, 'fake':0}

In [ ]:
def path_to_pixels(image_path):
    path = '/content/real_vs_fake/real-vs-fake/'
    im = Image.open(path+image_path)
    pixels = np.array(im.getdata()).flatten()
    return pixels

In [ ]:
df_train = df_train.repartition(npartitions=20)
output_path='train_data.csv'
for i in range(20):
  print(i)
  df_train1 = df_train.get_partition(i)
  df_train1['pixels'] = df_train1['path'].apply(path_to_pixels, meta=('path_to_pixels', object))
  df_train1.to_csv(output_path, mode='a', header=not os.path.exists(output_path))
!gsutil cp train_data.csv gs://paper-2-data/

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16


In [ ]:
df_val = df_val.repartition(npartitions=15)
output_path='val_data.csv'
for i in range(15):
  df_val1 = df_val.get_partition(i)
  df_val1['pixels'] = df_val1['path'].apply(path_to_pixels, meta=('path_to_pixels'))
  df_val1.to_csv(output_path, mode='a', header=not os.path.exists(output_path))
!gsutil cp val_data.csv gs://paper-2-data/